In [4]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the main directory containing subdirectories with images
main_dir = r'C:\Users\Plaksha\3D Objects\DL\fma_img\fma_img'

# Define image dimensions and batch size
img_height = 100
img_width = 100
batch_size = 16

# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use subset parameter for training data
)

validation_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use subset parameter for validation data
)

# Load the ResNet-50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the convolutional base
base_model.trainable = False

# Add a classification head
inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')(x)

# Compile the model
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

# Print accuracy and other performance metrics
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])
print("Training Loss:", history.history['loss'][-1])
print("Validation Loss:", history.history['val_loss'][-1])

# Rest of your code for tkinter UI, image preprocessing, and classification remains the same.


Found 230 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Epoch 1/20
15/15 [==============================] - 9s 401ms/step - loss: 2.4882 - accuracy: 0.0696 - val_loss: 2.3822 - val_accuracy: 0.1000
Epoch 2/20
15/15 [==============================] - 5s 308ms/step - loss: 2.3787 - accuracy: 0.0913 - val_loss: 2.3771 - val_accuracy: 0.1000
Epoch 3/20
15/15 [==============================] - 5s 302ms/step - loss: 2.3564 - accuracy: 0.0957 - val_loss: 2.3728 - val_accuracy: 0.1000
Epoch 4/20
15/15 [==============================] - 5s 355ms/step - loss: 2.3965 - accuracy: 0.0739 - val_loss: 2.3541 - val_accuracy: 0.1000
Epoch 5/20
15/15 [==============================] - 5s 327ms/step - loss: 2.3973 - accuracy: 0.0783 - val_loss: 2.3354 - val_accuracy: 0.1000
Epoch 6/20
15/15 [==============================] - 5s 308ms/step - loss: 2.3962 - accuracy: 0.0696 - val_loss: 2.3079 - val_accuracy: 0.1000
Epoch 7/20
15/15 [==============================] - 5s 335ms/step

In [5]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Set the path to the main directory containing subdirectories with images
main_dir = r'C:\Users\Plaksha\3D Objects\DL\fma_img\fma_img'

# Define image dimensions and batch size
img_height = 100
img_width = 100
batch_size = 16

# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use subset parameter for training data
)

validation_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use subset parameter for validation data
)

# Load the ResNet-50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the convolutional base
base_model.trainable = False

# Add a classification head
inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')(x)

# Compile the model
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

# Print accuracy and other performance metrics
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])
print("Training Loss:", history.history['loss'][-1])
print("Validation Loss:", history.history['val_loss'][-1])

# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to calculate embeddings
def calculate_embeddings(image_path):
    img_array = preprocess_image(image_path)
    embeddings = model.predict(img_array)
    return embeddings

# Function to open file dialog and get the image path
def browse_file():
    file_path = filedialog.askopenfilename()
    image_path_var.set(file_path)
    if file_path:
        img = Image.open(file_path)
        img = img.resize((200, 200), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img

# Function to match embeddings and determine class
def match_embeddings():
    image_path = image_path_var.get()
    if image_path:
        uploaded_embeddings = calculate_embeddings(image_path)
        min_distance = float('inf')
        min_class = None
        for subdir in os.listdir(main_dir):
            subdir_path = os.path.join(main_dir, subdir)
            for img_file in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, img_file)
                embeddings = calculate_embeddings(img_path)
                distance = np.linalg.norm(uploaded_embeddings - embeddings)
                if distance < min_distance:
                    min_distance = distance
                    min_class = subdir
        result_var.set(f"The song clip belongs to class: {min_class}")

# Create tkinter window
root = tk.Tk()
root.title("Image Classification")

# Create widgets
browse_button = tk.Button(root, text="Upload Image", command=browse_file)
browse_button.pack(pady=10)

image_label = tk.Label(root)
image_label.pack(pady=10)

image_path_var = tk.StringVar()
image_path_entry = tk.Entry(root, textvariable=image_path_var, state='readonly')
image_path_entry.pack(pady=10)

classify_button = tk.Button(root, text="Classify Image", command=match_embeddings)
classify_button.pack(pady=10)

result_var = tk.StringVar()
result_label = tk.Label(root, textvariable=result_var)
result_label.pack(pady=10)

# Function to close the tkinter window
def close_window():
    root.destroy()

# Bind closing event to close_window function
root.protocol("WM_DELETE_WINDOW", close_window)

# Run the tkinter event loop
root.mainloop()


Found 230 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Epoch 1/50
15/15 [==============================] - 10s 368ms/step - loss: 2.4636 - accuracy: 0.0826 - val_loss: 2.3936 - val_accuracy: 0.1000
Epoch 2/50
15/15 [==============================] - 4s 291ms/step - loss: 2.3854 - accuracy: 0.0913 - val_loss: 2.3278 - val_accuracy: 0.1000
Epoch 3/50
15/15 [==============================] - 4s 291ms/step - loss: 2.4048 - accuracy: 0.0826 - val_loss: 2.3354 - val_accuracy: 0.1000
Epoch 4/50
15/15 [==============================] - 4s 290ms/step - loss: 2.3935 - accuracy: 0.0870 - val_loss: 2.3709 - val_accuracy: 0.1000
Epoch 5/50
15/15 [==============================] - 4s 289ms/step - loss: 2.3813 - accuracy: 0.1000 - val_loss: 2.3279 - val_accuracy: 0.1000
Epoch 6/50
15/15 [==============================] - 4s 292ms/step - loss: 2.4003 - accuracy: 0.0478 - val_loss: 2.3433 - val_accuracy: 0.0800
Epoch 7/50
15/15 [==============================] - 4s 291ms/ste

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Plaksha\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Plaksha\AppData\Local\Temp\ipykernel_13708\3912792848.py", line 88, in browse_file
    img = img.resize((200, 200), Image.ANTIALIAS)
                                 ^^^^^^^^^^^^^^^
AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 57ms/step
